In [81]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
data = pd.read_csv('merged_data.csv')
data.head()

,avg_monthly_hrs,dept_id,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status
0,246.0,NaN,NaN,0.866838,6,NaN,medium,0.134415,Left,4.0,124467,NaN,NaN,42.0,Female,Married
1,134.0,NaN,NaN,0.555718,2,NaN,low,0.511041,Left,3.0,112210,NaN,NaN,23.0,Female,Unmarried
2,156.0,D00-SS,1.0,0.474082,2,NaN,medium,0.405101,Left,3.0,126150,Sales,Edward J Bayley,24.0,Female,Unmarried
3,256.0,D00-SP,NaN,0.961360,6,NaN,low,0.152974,Left,4.0,125346,Support,Amelia Westray,51.0,Female,Married
4,146.0,D00-SS,NaN,0.507349,2,NaN,medium,0.434845,Left,3.0,113707,Sales,Edward J Bayley,23.0,Female,Unmarried


# performing steps to clean the data

- TP in department  - has nt meaningful info, but few rows are blank
- Avg monthly hours has no blanks
- Employee ID has 5 zeros? removed
- Duplicate records- clean  - 
- Blank dept ID  707? others OT
- Filled complaint column all blanks converted to 0
- last_eval column- average rating for 0
- recently_promoted blanks to 0
- how to treat tenure above 10 years age less than 30 - 400 such people
- satisfaction blanks to average for now
- tenure blanks converted to 1

In [82]:
#pip install pandas-profiling
#profile = ProfileReport(data, title = 'Profile_report')
#profile

In [83]:
#tackling dept_id clumn... 5% of the data cannot be used hence dropping.
data=data[data['dept_id'].notna()]
data.shape  #707 rows removed

(13443, 16)

In [84]:
#tackling employee_id column removing those 5 rows with 0
data = data[data['employee_id'] != 0]  #only selecting those employee_id data which is not 0
data.shape

(13439, 16)

In [85]:
#tackling duplicate records
data = data.drop_duplicates()
data.shape

(13410, 16)

In [86]:
#tackling nans of filed_complaint column
data = pd.concat([data, pd.get_dummies(data['filed_complaint'], prefix = 'filed_complaint')], axis = 1)  #creating a new column using get_dummies
data.drop(['filed_complaint'], axis = 1, inplace = True) #dropped the original column
data.rename(columns ={'filed_complaint_1.0' : 'filed_complaint'}, inplace = True) #renamed the created column to the previous one
data.head()

,avg_monthly_hrs,dept_id,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status,filed_complaint
2,156.0,D00-SS,0.474082,2,NaN,medium,0.405101,Left,3.0,126150,Sales,Edward J Bayley,24.0,Female,Unmarried,1
3,256.0,D00-SP,0.961360,6,NaN,low,0.152974,Left,4.0,125346,Support,Amelia Westray,51.0,Female,Married,0
4,146.0,D00-SS,0.507349,2,NaN,medium,0.434845,Left,3.0,113707,Sales,Edward J Bayley,23.0,Female,Unmarried,0
5,135.0,D00-MT,0.482184,2,NaN,low,0.381545,Left,3.0,121685,Marketing,Reuben Swann,23.0,Female,Unmarried,0
6,270.0,D00-PD,0.867087,6,NaN,low,0.172575,Left,4.0,119912,Product,Darcy Staines,24.0,Female,Unmarried,0


In [87]:
#last_evaluation column  - all blanks to average rating
print(data['last_evaluation'].isnull().sum()) #figure correlates with the profiling report
data['last_evaluation'].fillna(data['last_evaluation'].mean(), inplace = True)
print(data['last_evaluation'].isnull().sum())  # we notice that now there are no nans
data.head() # a quick look at the data

1421
0


,avg_monthly_hrs,dept_id,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status,filed_complaint
2,156.0,D00-SS,0.474082,2,NaN,medium,0.405101,Left,3.0,126150,Sales,Edward J Bayley,24.0,Female,Unmarried,1
3,256.0,D00-SP,0.961360,6,NaN,low,0.152974,Left,4.0,125346,Support,Amelia Westray,51.0,Female,Married,0
4,146.0,D00-SS,0.507349,2,NaN,medium,0.434845,Left,3.0,113707,Sales,Edward J Bayley,23.0,Female,Unmarried,0
5,135.0,D00-MT,0.482184,2,NaN,low,0.381545,Left,3.0,121685,Marketing,Reuben Swann,23.0,Female,Unmarried,0
6,270.0,D00-PD,0.867087,6,NaN,low,0.172575,Left,4.0,119912,Product,Darcy Staines,24.0,Female,Unmarried,0


In [92]:
#tackling recently promoted_column - filling NaNs with 0
data['recently_promoted'] = data['recently_promoted'].fillna(0).apply(np.int64)
print(data['recently_promoted'].value_counts())  # we see that the data is skewed twoards not-promoted
data.head() #quick look at the data

0    13128
1      282
Name: recently_promoted, dtype: int64


,avg_monthly_hrs,dept_id,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status,filed_complaint
2,156.0,D00-SS,0.474082,2,0,medium,0.405101,Left,3.0,126150,Sales,Edward J Bayley,24.0,Female,Unmarried,1
3,256.0,D00-SP,0.961360,6,0,low,0.152974,Left,4.0,125346,Support,Amelia Westray,51.0,Female,Married,0
4,146.0,D00-SS,0.507349,2,0,medium,0.434845,Left,3.0,113707,Sales,Edward J Bayley,23.0,Female,Unmarried,0
5,135.0,D00-MT,0.482184,2,0,low,0.381545,Left,3.0,121685,Marketing,Reuben Swann,23.0,Female,Unmarried,0
6,270.0,D00-PD,0.867087,6,0,low,0.172575,Left,4.0,119912,Product,Darcy Staines,24.0,Female,Unmarried,0


In [94]:
#tackling the satisfaction column - treatment convert blanks to average
print(data['satisfaction'].isnull().sum())  #checking to see how many nulls
data['satisfaction'].fillna(data['satisfaction'].mean() , inplace = True) 
print(data['satisfaction'].isnull().sum()) #checking to see there are no nulls anymore
data.head() #quick look at the data

150
0


,avg_monthly_hrs,dept_id,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status,filed_complaint
2,156.0,D00-SS,0.474082,2,0,medium,0.405101,Left,3.0,126150,Sales,Edward J Bayley,24.0,Female,Unmarried,1
3,256.0,D00-SP,0.961360,6,0,low,0.152974,Left,4.0,125346,Support,Amelia Westray,51.0,Female,Married,0
4,146.0,D00-SS,0.507349,2,0,medium,0.434845,Left,3.0,113707,Sales,Edward J Bayley,23.0,Female,Unmarried,0
5,135.0,D00-MT,0.482184,2,0,low,0.381545,Left,3.0,121685,Marketing,Reuben Swann,23.0,Female,Unmarried,0
6,270.0,D00-PD,0.867087,6,0,low,0.172575,Left,4.0,119912,Product,Darcy Staines,24.0,Female,Unmarried,0


In [99]:
# tackling tenure blanks to be converted to 0
print(data['tenure'].isnull().sum())  #checking to see how many
data['tenure'].fillna(0, inplace = True)
print(data['tenure'].isnull().sum()) # all done
data.head()

0
0


,avg_monthly_hrs,dept_id,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status,filed_complaint
2,156.0,D00-SS,0.474082,2,0,medium,0.405101,Left,3.0,126150,Sales,Edward J Bayley,24.0,Female,Unmarried,1
3,256.0,D00-SP,0.961360,6,0,low,0.152974,Left,4.0,125346,Support,Amelia Westray,51.0,Female,Married,0
4,146.0,D00-SS,0.507349,2,0,medium,0.434845,Left,3.0,113707,Sales,Edward J Bayley,23.0,Female,Unmarried,0
5,135.0,D00-MT,0.482184,2,0,low,0.381545,Left,3.0,121685,Marketing,Reuben Swann,23.0,Female,Unmarried,0
6,270.0,D00-PD,0.867087,6,0,low,0.172575,Left,4.0,119912,Product,Darcy Staines,24.0,Female,Unmarried,0


In [102]:
data.info()
#seems like all the data points are tackled except for dept_head & dept_name

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13410 entries, 2 to 14120
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   avg_monthly_hrs    13410 non-null  float64
 1   dept_id            13410 non-null  object 
 2   last_evaluation    13410 non-null  float64
 3   n_projects         13410 non-null  int64  
 4   recently_promoted  13410 non-null  int64  
 5   salary             13410 non-null  object 
 6   satisfaction       13410 non-null  float64
 7   status             13410 non-null  object 
 8   tenure             13410 non-null  float64
 9   employee_id        13410 non-null  int64  
 10  dept_name          13203 non-null  object 
 11  dept_head          13203 non-null  object 
 12  age                13410 non-null  float64
 13  gender             13410 non-null  object 
 14  marital_status     13410 non-null  object 
 15  filed_complaint    13410 non-null  uint8  
dtypes: float64(5), int64(3

In [105]:
#lets take a closer look
data[data.isnull().any(axis = 1)]

,avg_monthly_hrs,dept_id,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,dept_head,age,gender,marital_status,filed_complaint
212,137.0,-IT,0.543217,2,0,medium,0.442070,Left,3.0,126341,NaN,NaN,24.0,Female,Unmarried,0
278,262.0,-IT,0.901248,5,0,low,0.717886,Left,5.0,117187,NaN,NaN,24.0,Female,Unmarried,0
367,134.0,-IT,0.508337,2,0,medium,0.456376,Left,3.0,124194,NaN,NaN,22.0,Female,Unmarried,0
410,267.0,-IT,0.785357,5,0,low,0.927001,Left,5.0,119515,NaN,NaN,22.0,Female,Unmarried,0
562,127.0,-IT,0.559907,2,0,medium,0.440033,Left,3.0,119346,NaN,NaN,25.0,Female,Unmarried,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13611,223.0,-IT,0.897100,5,0,medium,0.895458,Left,5.0,133677,NaN,NaN,40.0,Female,Married,0
13637,123.0,-IT,0.774735,2,0,medium,0.508124,Employed,4.0,107214,NaN,NaN,24.0,Female,Unmarried,0
13655,216.0,-IT,0.736369,4,0,medium,0.856901,Employed,4.0,116349,NaN,NaN,54.0,Male,Married,0
13862,128.0,-IT,0.458317,3,0,low,0.447364,Employed,2.0,106545,NaN,NaN,23.0,Male,Unmarried,1


In [106]:
#looking at department ID see the unique value
data['dept_id'].value_counts()
#it seems to me that all the data belongs to the IT depatment

D00-SS     3895
D00-ENG    2573
D00-SP     2108
D00-IT     1152
D00-PD      853
D00-MT      812
D00-FN      722
D00-MN      590
-IT         207
D00-AD      175
D00-PR      173
D00-TP      150
Name: dept_id, dtype: int64

In [112]:
#replacing data in dept_id column -IT with D00-IT
data.loc[data['dept_id'] == '-IT'] = 'D00-IT'  #locking on the data which as -IT and then replacing it with D00IT
data['dept_id'].value_counts()  #seems to be fine now

D00-SS     3895
D00-ENG    2573
D00-SP     2108
D00-IT     1359
D00-PD      853
D00-MT      812
D00-FN      722
D00-MN      590
D00-AD      175
D00-PR      173
D00-TP      150
Name: dept_id, dtype: int64

In [116]:
#need to tackle dept_name
print(data['dept_name'].value_counts())
#hence replacing all the nans with IT
data['dept_name'].replace(np.nan,'IT', inplace = True)
print('New IT -> ' (data['dept_name'] == 'IT').sum())  # it has increased

Sales          3895
Engineering    2573
Support        2108
IT             1152
Product         853
Marketing       812
Finance         722
Management      590
Admin           175
Procurement     173
Temp            150
Name: dept_name, dtype: int64
1359


In [119]:
#department head seems to have no relevance hence droppping the column as well as dept_id & employee_id
data.drop(['dept_head', 'dept_id', 'employee_id'], axis = 1, inplace = True)
data

,avg_monthly_hrs,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id,dept_name,age,gender,marital_status,filed_complaint
2,156.0,0.474082,2,0,medium,0.405101,Left,3.0,126150,Sales,24.0,Female,Unmarried,1
3,256.0,0.96136,6,0,low,0.152974,Left,4.0,125346,Support,51.0,Female,Married,0
4,146.0,0.507349,2,0,medium,0.434845,Left,3.0,113707,Sales,23.0,Female,Unmarried,0
5,135.0,0.482184,2,0,low,0.381545,Left,3.0,121685,Marketing,23.0,Female,Unmarried,0
6,270.0,0.867087,6,0,low,0.172575,Left,4.0,119912,Product,24.0,Female,Unmarried,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14116,141.0,0.537866,3,0,low,0.610841,Employed,3.0,100833,Sales,29.0,Male,Unmarried,0
14117,168.0,0.643553,3,0,low,0.489559,Employed,3.0,118090,Sales,24.0,Female,Unmarried,0
14118,257.0,0.718356,3,0,medium,0.944942,Employed,3.0,106064,Support,27.0,Male,Unmarried,0
14119,242.0,0.836603,4,0,low,0.740136,Employed,2.0,113083,IT,46.0,Male,Married,0


In [120]:
data.info()  # there are no missing values in the data any longer...yipee!
data.to_csv('clean_data.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13410 entries, 2 to 14120
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   avg_monthly_hrs    13410 non-null  object
 1   last_evaluation    13410 non-null  object
 2   n_projects         13410 non-null  object
 3   recently_promoted  13410 non-null  object
 4   salary             13410 non-null  object
 5   satisfaction       13410 non-null  object
 6   status             13410 non-null  object
 7   tenure             13410 non-null  object
 8   employee_id        13410 non-null  object
 9   dept_name          13410 non-null  object
 10  age                13410 non-null  object
 11  gender             13410 non-null  object
 12  marital_status     13410 non-null  object
 13  filed_complaint    13410 non-null  object
dtypes: object(14)
memory usage: 2.0+ MB


In [122]:
#making the data ready for machine learning - one hot encoding the categorical data
# recently_promoted, salary, status, dept_name, gender, marital_status, filed complaint
cols = ['recently_promoted', 'salary', 'status','dept_name','gender','marital_status','filed_complaint']
df = pd.get_dummies(data , columns = cols, drop_first = True)
df.head()

,avg_monthly_hrs,last_evaluation,n_projects,satisfaction,tenure,employee_id,age,recently_promoted_1,salary_low,salary_medium,...,dept_name_Management,dept_name_Marketing,dept_name_Procurement,dept_name_Product,dept_name_Sales,dept_name_Support,dept_name_Temp,gender_Male,marital_status_Unmarried,filed_complaint_1
2,156.0,0.474082,2,0.405101,3.0,126150,24.0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
3,256.0,0.96136,6,0.152974,4.0,125346,51.0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,146.0,0.507349,2,0.434845,3.0,113707,23.0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
5,135.0,0.482184,2,0.381545,3.0,121685,23.0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
6,270.0,0.867087,6,0.172575,4.0,119912,24.0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
